In [1]:
#Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", 101)

###Data Description

Column | Description
:---|:---
`id` | Record index
`timestamp` | Datetime (YYYY:MM:DD HH:MM:SS) when data was collected
`country` | Current country of employment
`employment_status` | Whether a candidate is Full time, Part time, Independent or freelancer or company owner
`job_title` | Current job title of the candidate
`job_years` | Total job experience (in Years)
`is_manager` | Whether the candidate holds a managerial position or not (Yes or No)
`hours_per_week` | No. of hours per day committed to the current job
`telecommute_days_per_week` | No. of telecommuting days per week (working from home)
`education` | The highest degree in education the candidate has received
`is_education_computer_related` | Is the education related to the field of computer science (Yes or No)
`certifications` | Does the candidate have any relevant certifications (Yes or No)
`salary` | Monthly Salary (in US $$)

In [2]:
# Dataset is already loaded below
data = pd.read_csv("employee.csv")

In [3]:
# Dimensions of training data
data.shape

(4277, 13)

In [4]:
# Print first few rows of data
data.head()

,id,timestamp,country,employment_status,job_title,job_years,is_manager,hours_per_week,telecommute_days_per_week,education,is_education_computer_related,certifications,salary
0,1,12/11/2018 10:52:26,Slovenia,Full time,Developer,4.78393,Yes,40.0,0.0,Bachelors (4 years),Yes,No,7187.743094
1,2,1/5/2017 16:57:50,United States,Full time,DBA,5.00000,No,40.0,5.0,Bachelors (4 years),Yes,No,10000.000000
2,3,12/18/2017 8:13:15,Sweden,Full time,DBA,1.00000,No,40.0,0.0,Masters,Yes,Yes,7000.000000
3,4,12/27/2018 4:56:52,United States,Full time,DBA,1.00000,No,40.0,2.0,Bachelors (4 years),No,No,8333.000000
4,5,12/11/2018 14:07:58,United States,Full time,Developer,3.00000,No,40.0,2.0,Masters,Yes,Yes,7137.000000


In [5]:
# drop id, timestamp and country columns
data = data.drop(columns=['id', 'timestamp','country'])

In [6]:
# Explore columns
data.columns

Index(['employment_status', 'job_title', 'job_years', 'is_manager',
       'hours_per_week', 'telecommute_days_per_week', 'education',
       'is_education_computer_related', 'certifications', 'salary'],
      dtype='object')

In [7]:
# replace NANs in hours_per_week with median value of the column
data.loc[data['hours_per_week'].isna(), 'hours_per_week'] = data['hours_per_week'].median()
data.loc[data['telecommute_days_per_week'].isna(), 'telecommute_days_per_week'] = data['telecommute_days_per_week'].median()

In [8]:
#Handling null values in categorical columns
data = data.dropna()

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4261 entries, 0 to 4276
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   employment_status              4261 non-null   object 
 1   job_title                      4261 non-null   object 
 2   job_years                      4261 non-null   float64
 3   is_manager                     4261 non-null   object 
 4   hours_per_week                 4261 non-null   float64
 5   telecommute_days_per_week      4261 non-null   float64
 6   education                      4261 non-null   object 
 7   is_education_computer_related  4261 non-null   object 
 8   certifications                 4261 non-null   object 
 9   salary                         4261 non-null   float64
dtypes: float64(4), object(6)
memory usage: 366.2+ KB


###Data Visualization :

## Feature Encoding and Normalization

Before training the model, we should perform one-hot encoding for all categorical/discrete variables, normalize continuous variables and then combine all data to form the training set.

In [10]:
# create another copy of dataset and append encoded features to it
data_train = data.copy()
data_train.head()

,employment_status,job_title,job_years,is_manager,hours_per_week,telecommute_days_per_week,education,is_education_computer_related,certifications,salary
0,Full time,Developer,4.78393,Yes,40.0,0.0,Bachelors (4 years),Yes,No,7187.743094
1,Full time,DBA,5.00000,No,40.0,5.0,Bachelors (4 years),Yes,No,10000.000000
2,Full time,DBA,1.00000,No,40.0,0.0,Masters,Yes,Yes,7000.000000
3,Full time,DBA,1.00000,No,40.0,2.0,Bachelors (4 years),No,No,8333.000000
4,Full time,Developer,3.00000,No,40.0,2.0,Masters,Yes,Yes,7137.000000


In [11]:
# select categorical features
cat_cols = [c for c in data_train.columns if data_train[c].dtype == 'object'
            and c not in ['is_manager', 'certifications']]
cat_data = data_train[cat_cols]
cat_cols

['employment_status',
 'job_title',
 'education',
 'is_education_computer_related']

In [12]:
#Encoding binary variables
binary_cols = ['is_manager', 'certifications']
for c in binary_cols:
    data_train[c] = data_train[c].replace(to_replace=['Yes'], value=1)
    data_train[c] = data_train[c].replace(to_replace=['No'], value=0)

In [13]:
final_data = pd.get_dummies(data_train, columns=cat_cols, drop_first= True,dtype=int)
final_data.shape

(4261, 25)

In [14]:
final_data.columns

Index(['job_years', 'is_manager', 'hours_per_week',
       'telecommute_days_per_week', 'certifications', 'salary',
       'employment_status_Independent or freelancer or company owner',
       'employment_status_Part time', 'job_title_Analytics consultant',
       'job_title_Architect', 'job_title_DBA', 'job_title_Data Scientist',
       'job_title_Developer', 'job_title_Engineer', 'job_title_Manager',
       'job_title_Other', 'job_title_Principal database engineer',
       'job_title_Sales', 'job_title_Sr Consultant ',
       'education_Bachelors (4 years)', 'education_Doctorate/PhD',
       'education_Masters', 'education_None (no degree completed)',
       'is_education_computer_related_Unknown',
       'is_education_computer_related_Yes'],
      dtype='object')

In [15]:
final_data

,job_years,is_manager,hours_per_week,telecommute_days_per_week,certifications,salary,employment_status_Independent or freelancer or company owner,employment_status_Part time,job_title_Analytics consultant,job_title_Architect,job_title_DBA,job_title_Data Scientist,job_title_Developer,job_title_Engineer,job_title_Manager,job_title_Other,job_title_Principal database engineer,job_title_Sales,job_title_Sr Consultant,education_Bachelors (4 years),education_Doctorate/PhD,education_Masters,education_None (no degree completed),is_education_computer_related_Unknown,is_education_computer_related_Yes
0,4.783930,1,40.0,0.0,0,7187.743094,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1
1,5.000000,0,40.0,5.0,0,10000.000000,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2,1.000000,0,40.0,0.0,1,7000.000000,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
3,1.000000,0,40.0,2.0,0,8333.000000,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,3.000000,0,40.0,2.0,1,7137.000000,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,2.000000,1,40.0,0.0,0,4917.000000,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
4273,4.140793,1,40.0,0.0,0,7033.845701,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4274,5.000000,0,40.0,0.0,0,7166.000000,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
4275,4.000000,1,40.0,2.0,0,9583.000000,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1


## Train Test Split

In [16]:
y = final_data['salary']
X = final_data.drop(columns=['salary'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print("Training Set Dimensions:", X_train.shape)
print("Validation Set Dimensions:", X_test.shape)

Training Set Dimensions: (3408, 24)
Validation Set Dimensions: (853, 24)


## Pre-processing data

### Standardization (Z-score normalization):

$$ x_{\text{std}} = \frac{x - \mu}{\sigma} $$

- **Purpose:** Standardization transforms the data to have a mean of 0 and a standard deviation of 1.
- **Properties:**
  - Centers the data around 0.
  - Rescales the data to have unit variance.
  - Does not bound the data within a specific range.
  - Preserves the shape of the distribution.
- **Use Cases:**
  - Algorithms that assume zero-centered data or require features to have a similar scale (e.g., gradient descent-based algorithms, support vector machines).
  - When the distribution of the features is Gaussian-like.

### Normalization (Min-Max scaling):

 $$ x_{\text{norm}} = \frac{x - \min(x)}{\max(x) - \min(x)} $$

- **Purpose:** Normalization scales the data to a fixed range, typically [0, 1] or [-1, 1].
- **Properties:**
  - Scales the data to a specified range.
  - Shifts the data to start at 0.
  - Does not affect the shape of the distribution.
  - Preserves the relative relationships between data points.
- **Use Cases:**
  - Neural networks, especially those with activation functions sensitive to input magnitudes (e.g., sigmoid or tanh functions).
  - When the distribution of the features is unknown or non-Gaussian.

**Choosing Between Standardization and Normalization:**
- Use standardization when the distribution of your features is approximately Gaussian-like and you want to center the data and rescale it to have unit variance.
- Use normalization when the scale of your features is important, or when you need to bound the features within a specific range.
- It's often beneficial to try both preprocessing techniques and evaluate their effects on model performance to determine which one works best for your specific dataset and model.


In [17]:
# select numerical features
num_cols = ['job_years','hours_per_week','telecommute_days_per_week']
num_cols


['job_years', 'hours_per_week', 'telecommute_days_per_week']

In [18]:
# Apply standard scaling on numeric data
scaler = StandardScaler()
scaler.fit(X_train[num_cols])
X_train[num_cols] = scaler.transform(X_train[num_cols])

In [19]:
X_train

,job_years,is_manager,hours_per_week,telecommute_days_per_week,certifications,employment_status_Independent or freelancer or company owner,employment_status_Part time,job_title_Analytics consultant,job_title_Architect,job_title_DBA,job_title_Data Scientist,job_title_Developer,job_title_Engineer,job_title_Manager,job_title_Other,job_title_Principal database engineer,job_title_Sales,job_title_Sr Consultant,education_Bachelors (4 years),education_Doctorate/PhD,education_Masters,education_None (no degree completed),is_education_computer_related_Unknown,is_education_computer_related_Yes
3728,2.785189,0,-2.192200,-0.619405,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3597,0.184998,0,0.032152,2.457756,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0
1344,-0.030303,0,0.042484,-0.619405,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
1771,0.271817,1,-0.490246,-0.003973,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1
2254,-1.738881,1,-2.192200,-0.619405,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1960,-0.001411,0,0.016986,-0.619405,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1
3421,0.156602,0,-0.030475,-0.619405,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1
2204,-1.236206,1,-0.490246,-0.619405,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1
998,-0.484426,0,0.784485,-0.619405,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0


In [20]:
#Fitting a Linear Regression Model
reg=LinearRegression()
reg.fit(X_train, y_train)

LinearRegression()

In [21]:
reg.coef_

array([ 1.96497395e+02,  3.63405800e+01,  1.82198234e+02,  2.11504926e+02,
        1.03323389e+02, -3.28011821e+01,  4.40667075e+01,  1.47792889e-12,
        1.07292568e+03,  8.21151565e+02,  8.08455817e+02,  5.12811268e+02,
        9.49363844e+02,  8.34541338e+02, -9.22224889e+00, -5.96855898e-13,
        6.87640243e+02,  3.55752020e+03,  3.43585797e+02,  4.54461698e+01,
        1.71118758e+02,  3.69448077e+02, -4.11373710e+02, -1.59039214e+02])

In [22]:
reg.intercept_

6171.524899969296

Just to recall

$\hat{y} = \alpha + \beta_1 * X_1 + \beta_2 * X_2 +...$

Our Final model is given by -

$\hat{y} = 6145.79 + 1.887 * X_1 + 7.22 * X_2 +...$


In [23]:
# Normalized MSE (Dividing by mean)
mean_squared_error(y_train,reg.predict(X_train))/np.mean(y_train)

210.6721622385044

In [24]:
# Predict on the test data
y_pred = reg.predict(X_test)

In [25]:
#Evaluate the model on test data
mse = mean_squared_error(y_pred, y_test)/np.mean(y_test)
print("Mean Squared Error:", mse)


Mean Squared Error: 10895.15428786776


*A lower MSE indicates that the model's predictions are closer to the actual values on average, while a higher MSE suggests larger errors between predictions and actual values.
❗Pre-processing on Test data not done.

In [26]:
#Q1. Preprocess Test data and get predictions

In [27]:
scaler.fit(X_test[num_cols])
X_test[num_cols] = scaler.transform(X_test[num_cols])

In [28]:
X_test

,job_years,is_manager,hours_per_week,telecommute_days_per_week,certifications,employment_status_Independent or freelancer or company owner,employment_status_Part time,job_title_Analytics consultant,job_title_Architect,job_title_DBA,job_title_Data Scientist,job_title_Developer,job_title_Engineer,job_title_Manager,job_title_Other,job_title_Principal database engineer,job_title_Sales,job_title_Sr Consultant,education_Bachelors (4 years),education_Doctorate/PhD,education_Masters,education_None (no degree completed),is_education_computer_related_Unknown,is_education_computer_related_Yes
3178,0.219048,0,-0.480564,2.217877,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
2551,-1.283623,0,2.429470,2.217877,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
656,0.219048,0,-0.480564,-0.654654,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2610,-0.152326,0,0.374697,1.068865,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
1710,-0.281842,0,-0.480564,-0.654654,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3136,0.219048,0,-0.480564,-0.654654,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
3336,0.219048,0,-0.480564,-0.654654,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
160,-1.784513,0,-0.480564,-0.654654,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3793,-0.411012,0,-0.227844,-0.654654,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1


In [29]:
y_hat = reg.predict(X_test)

In [30]:
#Predicted Values
y_hat

array([6949.87329873, 7618.2868697 , 7084.14043426, 7796.72798684,
       6677.37648848, 7599.90253562, 7168.24552772, 6734.8574543 ,
       6042.1659753 , 6321.48997733, 6235.55847573, 6388.10196503,
       7180.36129354, 6825.87865415, 6547.68948292, 8530.59688151,
       7313.89607308, 5966.93332098, 6961.44179982, 6459.43235677,
       6900.72096172, 6965.57584366, 7455.30437479, 6784.32822081,
       7293.93238284, 6472.50170908, 7155.13368945, 6640.16449975,
       6900.72096172, 7097.56825845, 7104.46068502, 7268.93174096,
       6903.92072438, 7918.14261827, 6843.75605703, 7186.94584772,
       6575.82509768, 8154.91530359, 7680.64729955, 7231.40343115,
       6488.50014655, 7448.11725147, 6968.12129688, 7499.62928207,
       7189.82794779, 7382.65685678, 7298.0003638 , 7570.81935534,
       7165.29704274, 7202.21713712, 7191.09216659, 6685.90457245,
       7555.82169847, 6792.76735892, 6685.90457245, 7603.09239475,
       6469.04574164, 7353.49242024, 6936.10508197, 7305.27323

In [31]:
# Normalized Mean Absolute Error (Dividing by mean)
mae_test= mean_absolute_error(y_test,y_hat)/np.mean(y_hat)
print("Test Mean Abosulte Error: ",mae_test)


Test Mean Abosulte Error:  0.11835507939743488


In [32]:
# Normalized Mean Squared Error (Dividing by mean)
mse_test = mean_squared_error(y_test,y_hat)/np.mean(y_hat)
print("Test Mean Squared Error: ",mse_test)


Test Mean Squared Error:  202.20592641178754
